This uses TeXSoup to segment and mark an example document.

Note: the paper uses a custom-built parser, however we intend to add our segmentation methods into TeXSoup in the future.

Be sure TeXSoup is installed: https://github.com/alvinwan/TexSoup#Installation (we recommend to install it from the GitHub source)

In [366]:
ds = ['\n', '\t', '\r', ' '] # all whitespaces
def split_keep(line, d = "\n"):
    ind = 0
    splits = []
    w = ''
    while ind < len(line):
        if line[ind] in ds:
            splits.append(w)
            splits.append(line[ind])
            ind += 1
            w = ''
        else: # add to word
            w += line[ind]
            ind += 1
    splits.append(w)
    #return splits[1:] # extra empty at top
    return splits

In [367]:
# tex filename?
tex_file = './example_tex/mnras_template_clean.tex'

tex_file_out = './example_tex/mnras_template_clean_marked.tex'

In [368]:
tex_lib = '/Users/jnaiman/TexSoup/'

from sys import path
path.append(tex_lib)

utils_lib = '../'
path.append(utils_lib)
from utils import get_tikz_preamble

#from TexSoup import TexSoup, tokens
import TexSoup

bodyMark = 'mainBody'
inlineMark = 'mainInline'

To allow markings with the tikz package:

In [369]:
preamble = get_tikz_preamble()
#preamble

In [370]:
with open(tex_file,'r') as f:
    tex_doc = f.read()

soup = TexSoup.TexSoup(tex_doc, tolerance=1)

In [371]:
texout = ''

markNum = 0
for isss, s in enumerate(soup.all):
    if type(s.expr) == TexSoup.data.TexNamedEnv:
        #import sys; sys.exit()
        # document?
        if 'begin' in s.expr.begin and 'document' in s.expr.begin:
            #import sys; sys.exit()
            # add preamble
            texout += preamble
            texout += '\\begin{document}\n'
            for ss in s.all:
                if type(ss.expr) == TexSoup.data.TexText: # extra for marking
                    #texout += ss.expr
                    if len(str(ss.expr).strip()) > 0: # not just white-space
                        if str(ss.expr).lstrip()[0] != '%': # not a comment:
                            splits = split_keep(ss.expr)
                            #import sys; sys.exit()
                            #texout += ss.expr
                            for sp in splits:
                                if len(sp) > 0 and sp not in ds:
                                    texout += '\\tikzmark{' + bodyMark + 'Start'+str(markNum)+'}'
                                    texout += sp
                                    texout += '\\tikzmark{' + bodyMark + 'End'+str(markNum)+'}'
                                    markNum += 1
                                else:
                                    texout += sp
                        else:
                            texout += ss.expr
                    else:
                        texout += ss.expr
                elif type(ss.expr) == TexSoup.data.TexMathModeEnv: # inline math
                    #print(type(ss.expr))
                    #print(ss.expr)
                    # add marks
                    texout += '\\tikzmark{' + inlineMark + 'Start'+str(markNum)+'}'
                    texout += str(ss.expr)
                    texout += '\\tikzmark{' + inlineMark + 'End'+str(markNum)+'}'
                    markNum += 1
                else: # not a pure text thing or math
                    #print(type(ss.expr))
                    texout += str(ss.expr)
            texout += '\\end{document}\n'
        else: # something else
            texout += str(s)
    else:
        texout += str(s)


In [372]:
with open(tex_file_out,'w') as f:
    print(texout,file=f,end='')

Compile to get aux file, in `./example_tex` run the following in the command line:

```bash
pdflatex -interaction=nonstopmode mnras_template_clean_marked.tex
```